In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/image-processing-thai-language-image-captioning/sample_submission.csv
/kaggle/input/image-processing-thai-language-image-captioning/capgen_v1.0_val.json
/kaggle/input/image-processing-thai-language-image-captioning/capgen_v1.0_train.json
/kaggle/input/image-processing-thai-language-image-captioning/val/val/travel/02371.jpg
/kaggle/input/image-processing-thai-language-image-captioning/val/val/travel/02360.jpg
/kaggle/input/image-processing-thai-language-image-captioning/val/val/travel/00266.jpg
/kaggle/input/image-processing-thai-language-image-captioning/val/val/travel/02450.jpg
/kaggle/input/image-processing-thai-language-image-captioning/val/val/travel/01496.jpg
/kaggle/input/image-processing-thai-language-image-captioning/val/val/travel/01600.jpg
/kaggle/input/image-processing-thai-language-image-captioning/val/val/travel/03508.jpg
/kaggle/input/image-processing-thai-language-image-captioning/val/val/travel/00614.jpg
/kaggle/input/image-processing-thai-language-image-c

In [4]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

In [5]:
!pip install deep-translator

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-6.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-6.7b",
    torch_dtype=torch.float16
).to(device)


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [24]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch
from deep_translator import GoogleTranslator
from PIL import ImageEnhance


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def translate_caption(text, dest='th'):
    try:
        translator = GoogleTranslator(source='en', target='th')
        return translator.translate(text)
    except:
        return "ไม่สามารถแปลได้"

def generate_caption(image_path):
    
    torch.cuda.empty_cache()

    image = Image.open(image_path).convert("RGB")
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.2)
    
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=50, num_beams=7, repetition_penalty=1.2)

    caption_en = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
    caption_th = translate_caption(caption_en)
    return caption_th


In [25]:
image_path = "/kaggle/input/image-processing-thai-language-image-captioning/test/test/00001.jpg"
caption_th = generate_caption(image_path)

In [26]:
print(caption_th)

จานสีขาวที่มีชามอาหารวางอยู่ด้านบน


In [27]:
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
import os

# Load sample submission
df_submission = pd.read_csv('/kaggle/input/image-processing-thai-language-image-captioning/sample_submission.csv')

# Initialize results storage
results = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def process_images_with_submission():
    # Initialize empty lists for results
    image_ids = []
    captions_th = []
    
    print(f"Processing {len(df_submission)} images...")
    
    for _, row in tqdm(df_submission.iterrows(), total=len(df_submission)):
        image_id = str(row['image_id']).zfill(5)
        image_path = f"/kaggle/input/image-processing-thai-language-image-captioning/test/test/{image_id}.jpg"
        
        print(f"\nProcessing image: {image_id}")
        try:
            caption_th = generate_caption(image_path)

            image_ids.append(image_id)
            captions_th.append(caption_th)
            
            print(f"ID: {image_id}")
            print(f"TH: {caption_th}")
            
        except Exception as e:
            print(f"Error processing {image_id}: {str(e)}")
            image_ids.append(image_id)
            captions_th.append("")

    submission_df = pd.DataFrame({
        'image_id': image_ids,
        'caption': captions_th
    })
    
    submission_df.to_csv('submission1.csv', index=False)
    return submission_df

final_submission = process_images_with_submission()

Processing 2000 images...


  0%|          | 0/2000 [00:00<?, ?it/s]


Processing image: 01354


  0%|          | 1/2000 [00:01<1:05:27,  1.96s/it]

ID: 01354
TH: ภาพระยะใกล้ของสิ่งมีชีวิตสีขาวขนาดเล็กบนใบไม้

Processing image: 01413


...

ID: 00195
TH: กลุ่มต้นไม้บนไหล่เขา

Processing image: 01026


100%|█████████▉| 1999/2000 [1:19:33<00:02,  2.41s/it]

ID: 01026
TH: ภาพระยะใกล้ของดอกไม้ที่กำลังเติบโตบนกิ่งไม้

Processing image: 01345


100%|██████████| 2000/2000 [1:19:35<00:00,  2.39s/it]

ID: 01345
TH: อาคารที่โรงแรมตั้งอยู่
